In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import threading
import re
import json
import html
import spacy
import pandas as pd
import concurrent.futures

In [ ]:

# Variable to control which website to crawl
selected_website = "whiskyauction"  # Change this to "website2" to select the second website
selected_type = 'test'              #Type test for testing, or other for global crawling

In [ ]:




def crawl_website1():
    # Base URL and header information
    url = "https://whiskyauction.com/wac/whiskyBrowserData"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
    }

    # Getting the total number of records
    params = {
        "start": 0,
        "length": 9,
        "currentsort": "auction_prose",
        "currentsortdirection": "DESC",
        "currentview": "card",
        "filterToBeApplied": 0
    }

    response = requests.post(url, headers=headers, data=params)
    soup = BeautifulSoup(response.text, 'html.parser')

    data_tmp = response.json()
    total = data_tmp['recordsTotal']

    # Calculate the total number of pages
    items_per_page = 100
    total_pages = total // items_per_page + (1 if total % items_per_page else 0)

    all_data = []
    fieldnames = ["Id", "Distillery", "Barrel", 'Brand', "Alcohol Strength", "Quantity", "Auction Date",
                  "Distilled Date",
                  "Bottled Date", "Age", "Type", "Packing", "Series", "Country", "Result"]

    def process_pages(start_page, end_page, lock):
        if selected_type =='test':
            end_page = 1
        for page in range(start_page, end_page):
            start = page * items_per_page
            data = {
                "start": start,
                "length": items_per_page,
                "currentsort": "auction_prose",
                "currentsortdirection": "DESC",
                "currentview": "card",
                "filterToBeApplied": 0
            }

            response = requests.post(url, headers=headers, data=data)
            data = response.json()
            page_data = data['data']

            # Parsing the data on each page
            page_results = []
            for item in page_data:
                whisky_info = {
                    "Id": item["item_id"],
                    "Distillery": item["distillery_prose"],
                    "Barrel": item["barrel"],
                    "Brand": item["bottler_label_prose"],
                    "Alcohol Strength": item["alcohol_strength"],
                    "Quantity": item["content_quantity"],
                    "Auction Date": item["auction_prose"],
                    "Distilled Date": item["distilled_date"],
                    "Bottled Date": item["bottled_date"],
                    "Age": item["age"],
                    "Type": item["whisky_type_prose"],
                    "Packing": item["packing_type_prose"],
                    "Series": item["series_prose"],
                    "Country": item["country_prose"],
                    "Result": item["result"]
                }
                page_results.append(whisky_info)

            # Use locks to ensure that only one thread writes to a file at a time
            with lock:
                with open('whisky_data.csv', 'a', newline='', encoding='utf-8') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    for info in page_results:
                        writer.writerow(info)

            time.sleep(2)

    # Creating a Lock Object
    lock = threading.Lock()

    # Create and start multiple threads
    num_threads = 10
    pages_per_thread = total_pages // num_threads

    threads = []
    for i in range(num_threads):
        start_page = i * pages_per_thread
        end_page = start_page + pages_per_thread if i != num_threads - 1 else total_pages
        thread = threading.Thread(target=process_pages, args=(start_page, end_page, lock))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    for info in all_data:
        print(info)




def crawl_website2():
    # List of stored URLs
    url_list = []

    for i in range(1, 92):
        surl = f"https://www.whiskyhammer.com/auction/past/auc-{i}"
        url_list.append(surl)
        print(url_list)
    html_list = url_list
    if selected_type == 'test':
        html_list = 'https://www.whiskyhammer.com/auction/past/auc-92'

    pagecount = 0
    data = []
    data_page = []
    count = 0
    count_attr = {'Distillery': 0, 'Age': 0, 'Country': 0, 'Bottles Produced': 0,
                  'Region': 0, 'Staus': 0, 'Whisky Type': 0, 'Size': 0,
                  'Strength': 0}
    with open('processed_data.txt', 'w') as f:
        pass

    def process_lot(lot_url):
        try:
            # Go to the lot page

            response = requests.get(lot_url)
            response.raise_for_status()  # Raise an exception if the response status is not 200

            distillery = lot_url.split('/')[5]
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract the required information
            result_head = soup.find('a', {"class": 'button alt'})
            Id = int(str(result_head).strip('<a class="button alt" href="/contact?_id=>Ask a question</a>'))

            result = soup.find_all('div', {'class': 'inner'})
            attr_list = []
            for tag in result:
                attr_list.append(tag.text.strip())

            temp = {'Id': Id, 'Distillery': distillery,
                    'Bottles Produced': 'not show',
                    'Region': 'not show', 'Staus': 'not show', 'Whisky Type': 'not show', 'Size': 'not show',
                    'Strength': 'not show'}

            attr_name = ["Bottles Produced", "Type", "Size", "Region",
                         'Strength']
            for attr in attr_list:
                for name in attr_name:

                    if "Distillery Status" in attr:
                        temp['Staus'] = attr
                        count_attr['Staus'] = count_attr['Staus'] + 1
                        break
                    if "Type" in attr:
                        temp['Whisky Type'] = attr
                        count_attr['Whisky Type'] = count_attr['Whisky Type'] + 1
                        break
                    if name in attr:
                        temp[name] = attr
                        count_attr[name] = count_attr[name] + 1
                        break

            distillery = temp["Distillery"]
            NumBottle = temp['Bottles Produced'].replace("Bottles Produced\n                                        ",
                                                         "")
            Type = temp['Whisky Type'].replace("Type\n                                        ", "")
            Amount = temp['Size'].replace("Size\n                                        ", "")
            region = temp['Region'].replace("Region\n                                        ", "")
            # country = temp['Country'].replace("Country", "")
            Staus = temp['Staus'].replace("Distillery Status\n                                        ", "")
            Strength = temp['Strength'].replace("Strength\n                                        ", "")

            # Add the data for this lot to the list
            data.append([Id, distillery, Staus, NumBottle, Type, Amount, Strength, region])
            for row in data:
                if len(row) != 8:  # 根据你的代码，我们假设数据应该有8列
                    print(row)
            print([Id, distillery, Staus, NumBottle, Type, Amount, Strength, region])
            with open('processed_data.txt', 'a') as f:
                f.write(f"{Id},{distillery},{Staus},{NumBottle},{Type},{Amount},{Strength},{region}\n")



        except requests.exceptions.RequestException as e:
            print(f"An error occurred while fetching {lot_url}: {e}")

    def extract_age_from_wine_name(wine_name):
        pattern = r'\b(\d+)\s*YEAR\s*OLD\b'
        match = re.search(pattern, wine_name, re.IGNORECASE)

        if match:
            age = int(match.group(1))
            return age
        else:
            return 'not shown'

    for htmls in html_list:
        with open('processed_data.txt', 'w') as f:
            pass
        pagecount = pagecount + 1
        print(pagecount)
        print("------------new month -------------")
        response_html = requests.get(htmls)
        html_soup = BeautifulSoup(response_html.text, 'html.parser')
        numLots = int(html_soup.find('div', {'class': 'numberOfProducts'}).text.strip(" \n").strip('Items'))
        pagenum = numLots // 50 + 1
        print(pagenum)
        lot_urls = []
        for page in range(1, pagenum + 1):
            page_url = htmls + "?page=" + str(page)
            print("start page" + str(page))
            response_page = requests.get(page_url)
            print('page' + str(page) + 'response')
            page_soup = BeautifulSoup(response_page.text, 'html.parser')
            page_text = str(page_soup)
            pattern = r':items="([^"]+)"'
            match = re.search(pattern, page_text)
            if match and match.group(1):
                items_attribute = match.group(1)
            else:
                continue

            decoded_items_attribute = html.unescape(items_attribute)
            auction_list = json.loads(decoded_items_attribute)

            for item in auction_list:
                url = item.get("url")
                lotid = item.get("id")
                name = item.get("name")
                highest_bid = item.get("item_price")
                time = item.get("ends")
                age = extract_age_from_wine_name(name)
                count_page_attr = [lotid, name, age, highest_bid, time]
                data_page.append([lotid, name, age, highest_bid, time])
                print(count_page_attr)
                if url:
                    lot_urls.append(url)
        lot_urls = list(set(lot_urls))

        batch_size = 50  # Setting the size of each batch

        for batch_start in range(0, len(lot_urls), batch_size):
            batch_urls = lot_urls[batch_start:batch_start + batch_size]

            with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(process_lot, lot_url) for lot_url in batch_urls]
                concurrent.futures.wait(futures)
        data = []

        # Read the processed data from the file and merge it.

        with open('processed_data.txt', 'r') as f:
            for line in f:
                values = line.strip().split(',')
                if len(values) == 8:
                    data.append(values)

        df = pd.DataFrame(data, columns=['Id', 'distillery', 'Staus', 'Bottles Produced', 'type',
                                         'amount',
                                         'Strength',
                                         'region'])

        df_page = pd.DataFrame(data_page, columns=['Id', 'name', 'vintage', 'highest_bid', 'time'])

        # Save the DataFrame to an Excel file
    df.to_csv('./whiskyhammer/auction_data'  + '.csv', index=False)
    df_page.to_csv('./whiskyhammer/auction_data_page_all' + '.csv', index=False)
    print(count)
    print(count_attr)

def data_transfer():
    new_columns = ["Id", "Barrel", "Distillery", "Brand", "Alcohol Strength", "Quantity", "Auction Date",
                   "Distilled Date", "Bottled Date", "Age", "Type", "Packing", "Series", "Country", "Result"]
    existing_data_path = 'whisky_data.csv'
    existing_df = pd.read_csv(existing_data_path, names=new_columns, header=None, skiprows=1)

    # Save the modified DataFrame back to the same file
    existing_df.to_csv(existing_data_path, index=False)

    nlp = spacy.load('en_core_web_sm')
    def detect_date(text):
        # 使用Spacy解析文本
        doc = nlp(text)
        # 提取日期实体
        for ent in doc.ents:
            if ent.label_ == 'DATE':
                try:
                    # 尝试将识别到的日期转换为所需格式
                    return pd.to_datetime(ent.text).strftime('%Y-%m')
                except:
                    pass
        return None

    # 读取数据
    df = pd.read_csv('whisky_data.csv')

    date_columns = ['Auction Date', 'Distilled Date', 'Bottled Date']
    for col in date_columns:
        df[col] = df[col].apply(lambda x: detect_date(str(x)) if pd.notnull(x) else x)

    # 保存更新后的数据
    df.to_csv('whisky_data.csv', index=False)

    # Fill missing Auction Date with '2020-12'
    df['Auction Date'].fillna('2020-12', inplace=True)

    # Function to process Age
    def process_age(row):
        if pd.isna(row['Age']) or row['Age'] == 0:
            if pd.notna(row['Bottled Date']) and pd.notna(row['Distilled Date']):
                year_bottled = int(row['Bottled Date'].split('-')[0])
                year_distilled = int(row['Distilled Date'].split('-')[0])
                return year_bottled - year_distilled
        return row['Age']

    # Function to process Bottled Date and Distilled Date
    def process_dates(row):
        if pd.isna(row['Bottled Date']) and pd.isna(row['Distilled Date']) and (
                pd.notna(row['Age']) and row['Age'] != 0):
            if isinstance(row['Auction Date'], str) and '-' in row['Auction Date']:
                year_auction = int(row['Auction Date'].split('-')[0])
                year_calculated = year_auction - int(row['Age'])
                date_str = str(year_calculated) + '-' + row['Auction Date'].split('-')[1]
                row['Bottled Date'] = row['Distilled Date'] = date_str
        return row

    # Apply functions to process data
    df['Age'] = df.apply(process_age, axis=1)
    df = df.apply(process_dates, axis=1)

    # Drop rows with missing Bottled Date, Distilled Date, and Age (or Age is 0)
    df.drop(df[(df['Bottled Date'].isna() & df['Distilled Date'].isna() & (df['Age'].isna() | df['Age'] == 0))].index,
            inplace=True)
    # Drop rows with missing Bottled Date, Distilled Date, and Age
    df.drop(df[(df['Bottled Date'].isna() & df['Distilled Date'].isna() & df['Age'].isna())].index, inplace=True)

    # Fill missing Packing values
    df['Packing'].fillna('Bottle', inplace=True)

    # Drop the 'Series' column
    df.drop(columns=['Series'], inplace=True)

    # Save processed data
    df.to_csv('processed_data.csv', index=False)

    # Load data
    df = pd.read_csv('processed_data.csv')
    # Drop rows with missing values
    df.dropna(inplace=True)

    # Drop rows where Alcohol Strength or Quantity is missing
    df.dropna(subset=['Alcohol Strength', 'Quantity'], inplace=True)

    # Fill Distilled Date when Bottled Date and Age are present
    def fill_distilled_date(row):
        if pd.notna(row['Bottled Date']) and pd.notna(row['Age']) and pd.isna(row['Distilled Date']):
            year_bottled = int(row['Bottled Date'].split('-')[0])
            year_calculated = year_bottled - int(row['Age'])
            return str(year_calculated) + '-' + row['Bottled Date'].split('-')[1]
        return row['Distilled Date']

    df['Distilled Date'] = df.apply(fill_distilled_date, axis=1)

    # Fill Bottled Date when Distilled Date and Age are present
    def fill_bottled_date(row):
        if pd.notna(row['Distilled Date']) and pd.notna(row['Age']) and pd.isna(row['Bottled Date']):
            year_distilled = int(row['Distilled Date'].split('-')[0])
            year_calculated = year_distilled + int(row['Age'])
            return str(year_calculated) + '-' + row['Distilled Date'].split('-')[1]
        return row['Bottled Date']

    df['Bottled Date'] = df.apply(fill_bottled_date, axis=1)


    # Check for missing values
    missing_data = df.isna().sum()

    # Output columns with missing values
    print("Columns with missing values:")
    for column, missing_count in missing_data.items():
        if missing_count > 0:
            print(f"{column}: {missing_count} missing values")

    # Load CSV file
    csv_file_path = 'processed_data.csv'  # Replace with your CSV data file path
    data_frame = pd.read_csv(csv_file_path)

    # Drop the 'Distilled Date' and 'Bottled Date' columns
    data_frame = data_frame.drop(columns=['Distilled Date'])
    data_frame = data_frame.drop(columns=['Bottled Date'])

    # Save the processed data as a CSV file
    output_csv_file_path = 'processed_data_new.csv'  # Replace with the desired path to save the CSV file
    data_frame.to_csv(output_csv_file_path, index=False)
    # Load existing data, skipping the first row
    existing_df = pd.read_csv('processed_data.csv', skiprows=1)

    # Save the modified DataFrame back to the same file
    existing_df.to_csv('processed_data.csv', index=False)
    print("Processing completed!")


if selected_website == "whiskyauction":
    crawl_website1()
    data_transfer()
elif selected_website == "whiskyhammer":
    crawl_website2()
else:
    print("Invalid selection. Please choose 'website1' or 'website2'.")
